In [ ]:
from fastapi import FastAPI, HTTPException, Form
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import psycopg2
import re
import os

app = FastAPI()

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:5174"],  # Allow your frontend's origin
    allow_credentials=True,
    allow_methods=["*"],  # Allow all HTTP methods (GET, POST, etc.)
    allow_headers=["*"],  # Allow all headers
)

# PostgreSQL connection configuration
DB_CONFIG = {
    'dbname': 'RE_DB',
    'user': 'postgres',
    'password': 'ykam2002',
    'host': 'localhost',
    'port': '5432'
}

# Database functions
def execute_query(query, params=None, fetch=False):
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()
    try:
        cursor.execute(query, params)
        if fetch:
            result = cursor.fetchall()
        else:
            conn.commit()
            result = None
    finally:
        cursor.close()
        conn.close()
    return result

# Initialize database table
def init_db():
    query = """
    CREATE TABLE IF NOT EXISTS regex_rules (
        id SERIAL PRIMARY KEY,
        re_pattern TEXT NOT NULL,
        replacement TEXT NOT NULL
    );
    """
    execute_query(query)

init_db()

class RegexRule(BaseModel):
    re_pattern: str
    replacement: str = ""

class LogContent(BaseModel):
    log_content: str

@app.post("/api/open-folder")
async def open_folder(folder_path: str = Form(...)):
    if not os.path.isdir(folder_path):
        raise HTTPException(status_code=400, detail="Invalid folder path")

    logs = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            with open(os.path.join(folder_path, file_name), 'r') as file:
                logs.append(file.read())

    return {"logs": logs}

@app.get("/api/regex")
async def get_regex():
    query = "SELECT id, re_pattern, replacement FROM regex_rules"
    rules = execute_query(query, fetch=True)
    return {"rules": [dict(zip(['id', 're_pattern', 'replacement'], rule)) for rule in rules]}

@app.post("/api/regex")
async def add_regex(rule: RegexRule):
    if not rule.re_pattern:
        raise HTTPException(status_code=400, detail="'re_pattern' is required")

    query = "INSERT INTO regex_rules (re_pattern, replacement) VALUES (%s, %s)"
    execute_query(query, (rule.re_pattern, rule.replacement))
    return {"message": "Regex rule added successfully"}

@app.delete("/api/regex/{rule_id}")
async def delete_regex(rule_id: int):
    query = "DELETE FROM regex_rules WHERE id = %s"
    execute_query(query, (rule_id,))
    return {"message": "Regex rule deleted successfully"}

@app.delete("/api/regex")
async def delete_all_regex():
    query = "DELETE FROM regex_rules"
    execute_query(query)
    return {"message": "All regex rules deleted successfully"}

@app.put("/api/regex/{rule_id}")
async def modify_regex(rule_id: int, rule: RegexRule):
    if not rule.re_pattern:
        raise HTTPException(status_code=400, detail="'re_pattern' is required")

    query = "UPDATE regex_rules SET re_pattern = %s, replacement = %s WHERE id = %s"
    execute_query(query, (rule.re_pattern, rule.replacement, rule_id))
    return {"message": "Regex rule updated successfully"}

@app.post("/api/process-log")
async def process_log(log: LogContent):
    if not log.log_content:
        raise HTTPException(status_code=400, detail="'log_content' is required")

    query = "SELECT re_pattern, replacement FROM regex_rules"
    rules = execute_query(query, fetch=True)

    log_content = log.log_content
    matches = []
    for re_pattern, replacement in rules:
        for match in re.finditer(re_pattern, log_content):
            matches.append({"match": match.group(), "start": match.start(), "end": match.end()})
        log_content = re.sub(re_pattern, replacement, log_content)

    return {"processed_log": log_content, "matches": matches}

@app.post("/api/save-log")
async def save_cleaned_log(log: LogContent):
    if not log.log_content:
        raise HTTPException(status_code=400, detail="'log_content' is required")

    # Define the path to the 'cleaned logs' folder on the desktop
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    cleaned_logs_path = os.path.join(desktop_path, "cleaned logs")

    # Create the folder if it doesn't exist
    if not os.path.exists(cleaned_logs_path):
        os.makedirs(cleaned_logs_path)

    # Save the log to a new file in the folder
    log_file_path = os.path.join(cleaned_logs_path, "cleaned_log.txt")
    with open(log_file_path, "w") as log_file:
        log_file.write(log.log_content)

    return {"message": "Cleaned log saved successfully", "file_path": log_file_path}


# To access Swagger documentation, go to: http://localhost:8000/docs or http://localhost:8000/redoc
if __name__ == "__main__":
    import nest_asyncio
    import uvicorn

    # Apply the patch to allow asyncio.run within Jupyter Notebook
    nest_asyncio.apply()

    # Start the server
    uvicorn.run(app, host="localhost", port=8000)


INFO:     Started server process [11496]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)


INFO:     ::1:60576 - "GET /api/regex HTTP/1.1" 200 OK
INFO:     ::1:60576 - "GET /api/regex HTTP/1.1" 200 OK
INFO:     ::1:60749 - "OPTIONS /api/process-log HTTP/1.1" 200 OK
INFO:     ::1:60749 - "POST /api/process-log HTTP/1.1" 400 Bad Request
INFO:     ::1:60962 - "POST /api/process-log HTTP/1.1" 200 OK
INFO:     ::1:60962 - "OPTIONS /api/save-log HTTP/1.1" 200 OK
INFO:     ::1:60962 - "POST /api/save-log HTTP/1.1" 200 OK
INFO:     ::1:61060 - "POST /api/process-log HTTP/1.1" 200 OK
INFO:     ::1:61060 - "POST /api/process-log HTTP/1.1" 200 OK
INFO:     ::1:61060 - "POST /api/save-log HTTP/1.1" 200 OK
INFO:     ::1:61232 - "GET /api/regex HTTP/1.1" 200 OK
INFO:     ::1:61232 - "GET /api/regex HTTP/1.1" 200 OK
INFO:     ::1:61662 - "GET /api/regex HTTP/1.1" 200 OK
INFO:     ::1:61662 - "GET /api/regex HTTP/1.1" 200 OK
INFO:     ::1:61667 - "GET /api/regex HTTP/1.1" 200 OK
INFO:     ::1:61667 - "GET /api/regex HTTP/1.1" 200 OK
INFO:     ::1:61717 - "GET /api/regex HTTP/1.1" 200 OK
INF